In [35]:
import os, json
import pandas as pd

path = '../../datasets/aifdb/TechnologyUse'
datasets = [path + '/' + ds for ds in os.listdir(path) if not ds.startswith('.')]
#print(datasets)
json_files = []
for path_to_json in datasets:
    print(path_to_json)
    if path_to_json.endswith('.json'):
        json_files.append(path_to_json)
print(len(json_files))  # for me this prints ['foo.json']

../../datasets/aifdb/TechnologyUse/nodeset5720.json
../../datasets/aifdb/TechnologyUse/nodeset5665.json
../../datasets/aifdb/TechnologyUse/nodeset5777.json
../../datasets/aifdb/TechnologyUse/nodeset5790.txt
../../datasets/aifdb/TechnologyUse/nodeset5798.json
../../datasets/aifdb/TechnologyUse/nodeset5753.txt
../../datasets/aifdb/TechnologyUse/nodeset5736.json
../../datasets/aifdb/TechnologyUse/nodeset5673.json
../../datasets/aifdb/TechnologyUse/nodeset5809.json
../../datasets/aifdb/TechnologyUse/nodeset5801.txt
../../datasets/aifdb/TechnologyUse/nodeset5825.json
../../datasets/aifdb/TechnologyUse/nodeset5746.txt
../../datasets/aifdb/TechnologyUse/nodeset5785.txt
../../datasets/aifdb/TechnologyUse/nodeset5686.json
../../datasets/aifdb/TechnologyUse/nodeset5787.txt
../../datasets/aifdb/TechnologyUse/nodeset5794.json
../../datasets/aifdb/TechnologyUse/nodeset5690.json
../../datasets/aifdb/TechnologyUse/nodeset5741.json
../../datasets/aifdb/TechnologyUse/nodeset5716.json
../../datasets/aif

In [36]:
import pandas as pd
from itertools import product

# CA = conflict
# RA = support
# TA = ?

result = []


for index, js in enumerate(json_files):
    args = []
    edges_list = []
    print(js)
    with open(js) as json_file:
        json_text = json.load(json_file)
        
        # Store nodes in a dataframe
        nodes = json_text['nodes']
        for node in nodes:
            args.append([node['nodeID'], node['type'], node['text']])
        nodes_df = pd.DataFrame(args, columns = ["nodeID", "type", "text"])

        # Store edges in a dataframe
        edges = json_text['edges']
        for edge in edges:
            edges_list.append([edge['edgeID'], edge['fromID'], edge['toID']])
        edges_df = pd.DataFrame(edges_list, columns = ["edgeID", "fromID", "toID"])
        
        arg_nodes = nodes_df.loc[nodes_df["type"].values == "I"]
        
        relationship_nodes = nodes_df.loc[(nodes_df["type"].values == "RA") | (nodes_df["type"].values == "CA")]
        
        for index, arg_row in arg_nodes.iterrows():
            node_child_edges = edges_df.loc[edges_df["fromID"] == arg_row["nodeID"]]
            for index, edge_row in node_child_edges.iterrows():
                if edge_row["toID"] in relationship_nodes["nodeID"].values:
                    # Locate with node
                    rel_node = relationship_nodes.loc[relationship_nodes["nodeID"] == edge_row["toID"]]
                    if len(rel_node["type"].values) > 0:
                        relationship = rel_node["type"].values[0]
                        # Get output node of relationship
                        out_edge = edges_df.loc[edges_df["fromID"].values == rel_node["nodeID"].values]
                        if len(out_edge["toID"].values) > 0:
                            out_node_ID = out_edge["toID"].values[0]
                            # get arguments from IDs
                            out_node_text = arg_nodes.loc[arg_nodes["nodeID"] == out_node_ID]["text"]
                            if len(out_node_text.values) > 0:
                                in_node_text = "\"" + arg_row["text"] + "\""
                                out_node_text = "\"" + out_node_text.values[0] + "\""
                                result.append([in_node_text, out_node_text, relationship])                            
                    
        
result_df = pd.DataFrame(result, columns = ["argument1", "argument2", "relationship"])
                
print(result_df)
                    

../../datasets/aifdb/TechnologyUse/nodeset5720.json
../../datasets/aifdb/TechnologyUse/nodeset5665.json
../../datasets/aifdb/TechnologyUse/nodeset5777.json
../../datasets/aifdb/TechnologyUse/nodeset5798.json
../../datasets/aifdb/TechnologyUse/nodeset5736.json
../../datasets/aifdb/TechnologyUse/nodeset5673.json
../../datasets/aifdb/TechnologyUse/nodeset5809.json
../../datasets/aifdb/TechnologyUse/nodeset5825.json
../../datasets/aifdb/TechnologyUse/nodeset5686.json
../../datasets/aifdb/TechnologyUse/nodeset5794.json
../../datasets/aifdb/TechnologyUse/nodeset5690.json
../../datasets/aifdb/TechnologyUse/nodeset5741.json
../../datasets/aifdb/TechnologyUse/nodeset5716.json
../../datasets/aifdb/TechnologyUse/nodeset5700.json
../../datasets/aifdb/TechnologyUse/nodeset5701.json
../../datasets/aifdb/TechnologyUse/nodeset5717.json
../../datasets/aifdb/TechnologyUse/nodeset5740.json
../../datasets/aifdb/TechnologyUse/nodeset5783.json
../../datasets/aifdb/TechnologyUse/nodeset5824.json
../../datase

In [37]:
attack_count = result_df[result_df['relationship'] == 'CA'].count().iloc[0]
print(attack_count)

277


In [38]:
support_count = result_df[result_df['relationship'] == 'RA'].count().iloc[0]
print(support_count)

552


In [39]:
neutral_count = result_df[result_df['relationship'] == 'TA'].count().iloc[0]
print(neutral_count)

0


In [40]:
# Replace aifdb relationship codes with our own
node_attack_support_codes = ["RA", "CA"]

result_df.loc[result_df["relationship"] == "RA", "relationship"] = "s"
result_df.loc[result_df["relationship"] == "CA", "relationship"] = "a"

print(result_df)

                                             argument1  \
0    "it is the first virtual reality headset to in...   
1    "whether a person who causes these problems wi...   
2    "User's place in the Web is becoming significa...   
3    "Since its origin, the Web had for principle t...   
4    "We're now facing a new trend. The ultimate st...   
..                                                 ...   
824  "physical storage options may not always be co...   
825  "But there are still privacy issues concerning...   
826  "Additionally, uploading and downloading from ...   
827  "Many social media apps can take over peoples ...   
828  "Being able to contact family and friends that...   

                                             argument2 relationship  
0    "The real breakthroughs in technology come in ...            s  
1    "The real breakthroughs in technology come in ...            a  
2    "Aspect of the Web in the near future will cha...            s  
3    "User's place in t

In [41]:
# Create neutral relationships

n_pairs = []


for i in range(attack_count):
    sample = result_df.sample(n=2)
    text1 = sample.iloc[0]["argument1"]
    text2 = sample.iloc[1]["argument2"]
    
    pair = result_df.loc[(result_df["argument1"] == text1) & (result_df["argument2"] == text2)]
    
    
    if len(pair) == 0:
        n_pairs.append([text1, text2, "n"])

n_pairs_df = pd.DataFrame(n_pairs, columns = ["argument1", "argument2", "relationship"])
result_df = pd.concat([result_df, n_pairs_df], ignore_index=True)
print(result[0])

['"it is the first virtual reality headset to include crisp, bright graphics"', '"The real breakthroughs in technology come in the form of things like virtual reality, brain mapping and robotic engineering."', 'RA']


In [42]:
attack_count = result_df[result_df['relationship'] == 'a'].count().iloc[0]
support_count = result_df[result_df['relationship'] == 's'].count().iloc[0]
n_count = result_df[result_df['relationship'] == 'n'].count().iloc[0]
print(attack_count, support_count, n_count)

277 552 275


In [43]:
# Unabstract arguments

result_df.to_csv('model_training/training_data/TechnologyUse.csv')

In [8]:
import pandas as pd
df = pd.read_csv('../../datasets/IBM_Debater_(R)_arg_quality_rank_30k/arg_quality_rank_30k.csv')
df["relationship"] = ""
th_element = df.sort_values(by=['WA']).iloc[10000]
threshold = th_element.WA
print(threshold)

#df.loc[df["stance_WA"] == 1].count()
#df.loc[df["stance_WA"] == -1].count()

0.740733152


In [9]:
df.head(10000)

,argument,topic,set,WA,MACE-P,stance_WA,stance_WA_conf,relationship
0,"""marriage"" isn't keeping up with the times. a...",We should abandon marriage,train,0.846165,0.297659,1,1.000000,
1,.a multi-party system would be too confusing a...,We should adopt a multi-party system,train,0.891271,0.726133,-1,1.000000,
2,\ero-tolerance policy in schools should not be...,We should adopt a zero-tolerance policy in sch...,dev,0.721192,0.396953,-1,1.000000,
3,`people reach their limit when it comes to the...,Assisted suicide should be a criminal offence,train,0.730395,0.225212,-1,1.000000,
4,"100% agree, should they do that, it would be a...",We should abolish safe spaces,train,0.236686,0.004104,1,0.805517,
...,...,...,...,...,...,...,...,...
9995,it can be used to prevent a child being mental...,We should ban cosmetic surgery for minors,train,0.727881,0.368241,-1,1.000000,
9996,it can be very beneficial for people with cert...,We should legalize cannabis,test,0.890397,0.713004,1,1.000000,
9997,it can be very educational and full of facts,We should abandon television,train,0.699708,0.249363,-1,1.000000,
9998,it can be viewed as undemocratic to force peop...,We should introduce compulsory voting,train,0.801551,0.401942,-1,1.000000,


In [10]:
df = df[["argument", "topic", "stance_WA", "WA"]]
translations = {
    -1: "a",
    1: "s"
}
df.loc[df["stance_WA"] == 1, "relationship"] = "s"
df.loc[df["stance_WA"] == -1, "relationship"] = "a"
df.loc[df["WA"] < threshold, "relationship"] = "n"
df.loc[df["relationship"] == "n"]

,argument,topic,stance_WA,WA,relationship
2,\ero-tolerance policy in schools should not be...,We should adopt a zero-tolerance policy in sch...,-1,0.721192,n
3,`people reach their limit when it comes to the...,Assisted suicide should be a criminal offence,-1,0.730395,n
4,"100% agree, should they do that, it would be a...",We should abolish safe spaces,1,0.236686,n
5,6 million people died so to deny the holocaust...,Holocaust denial should be a criminal offence,1,0.738337,n
7,a bad score in an intelligence test is a blow ...,Intelligence tests bring more harm than good,1,0.638716,n
...,...,...,...,...,...
30447,zoos provide a unique experience for children ...,We should abolish zoos,-1,0.667484,n
30464,zoos say they help repopulate dying population...,We should abolish zoos,1,0.735094,n
30466,zoos serve to imprison animals and show them o...,We should abolish zoos,1,0.729705,n
30474,zoos should be abolished for the animals to li...,We should abolish zoos,1,0.735364,n


In [11]:
df = df[["argument", "topic", "relationship"]]
df.columns = ["argument1", "argument2", "relationship"]
df


,argument1,argument2,relationship
0,"""marriage"" isn't keeping up with the times. a...",We should abandon marriage,s
1,.a multi-party system would be too confusing a...,We should adopt a multi-party system,a
2,\ero-tolerance policy in schools should not be...,We should adopt a zero-tolerance policy in sch...,n
3,`people reach their limit when it comes to the...,Assisted suicide should be a criminal offence,n
4,"100% agree, should they do that, it would be a...",We should abolish safe spaces,n
...,...,...,...
30492,zoos trap animals into a meaningless life only...,We should abolish zoos,s
30493,zoos treat animals badly they should be closed...,We should abolish zoos,s
30494,zoos unfairly imprison animals and cause them ...,We should abolish zoos,s
30495,Zoos work as educational centers and are not t...,We should abolish zoos,a


In [12]:
df = df.to_csv("IBM30K_with_n.csv")